### Import Required Libraries and Set Up Environment Variables

In [84]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [131]:
# Set environment variables from the .env in the local environment
load_dotenv('environment.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

print(type(nyt_api_key))


<class 'str'>


### Access the New York Times API

In [246]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"


# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
# filter_query = 'blockbuster'
# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
# field_list 
response_field = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# https://api.nytimes.com/svc/search/v2/articlesearch.json?q=election&api-key=yourkey
# https://developer.nytimes.com/docs/articlesearch-product/1/routes/articlesearch.json/get

# query_url = f"{url}&fq={filter_query}&api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"

query_url = f"{url}" \
            f"&api-key={nyt_api_key}" \
            f"&fq={filter_query}" \
            f"&begin_date={begin_date}" \
            f"&end_date={end_date}" \
            f"&sort={sort}"\
            f"&fl={response_field}"

            # f"q={query}" \
            # f"&page={page}" \


In [247]:
# Make a Request and check response 
response = requests.get(query_url)
response

<Response [200]>

In [249]:

reviews_list = []           # list to store the reviews
review_count = 0;      
     
for page in range(0, 20):    # loop through pages 0-19
# create query with a page number; API results show 10 articles at a time
    query_url_by_page = f"{query_url}&page={str(page)}"

# Make a "GET" request and Retrieve Articles (retrieve the JSON)
    try:
# Request Reviews by page using URL Query above
        reviews = requests.get(query_url_by_page).json()    
# Loop through the response & save/append each article to the reviews_list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
            review_count += 1
# Print page that was just retrieved
        print(f'Successfully obtained {review_count} reviews, page {page}')
# If there is page number w/ no results, print page # and break from loop
    except:
        print(f'Failed to obtain reviews, page {page}')
        break

# Add 12 sec b/w queries to stay within API query limits
    time.sleep(12)

Successfully obtained 10 reviews, page 0
Sleeping b/w queries...zzzz
Successfully obtained 20 reviews, page 1
Sleeping b/w queries...zzzz
Successfully obtained 30 reviews, page 2
Sleeping b/w queries...zzzz
Successfully obtained 40 reviews, page 3
Sleeping b/w queries...zzzz
Successfully obtained 50 reviews, page 4
Sleeping b/w queries...zzzz
Successfully obtained 60 reviews, page 5
Sleeping b/w queries...zzzz
Successfully obtained 70 reviews, page 6
Sleeping b/w queries...zzzz
Successfully obtained 80 reviews, page 7
Sleeping b/w queries...zzzz
Successfully obtained 90 reviews, page 8
Sleeping b/w queries...zzzz
Successfully obtained 100 reviews, page 9
Sleeping b/w queries...zzzz
Successfully obtained 110 reviews, page 10
Sleeping b/w queries...zzzz
Successfully obtained 120 reviews, page 11
Sleeping b/w queries...zzzz
Successfully obtained 130 reviews, page 12
Sleeping b/w queries...zzzz
Successfully obtained 140 reviews, page 13
Sleeping b/w queries...zzzz
Successfully obtained 150

In [262]:
#  Print sample of reviews added to reviews_list
count = 0
print(f'Total reviews obtained: {len(reviews_list)}' )
for r in reviews_list:
    count +=1
    print(f"Sample review ({count}): {r['headline']['main']}") #snippet
    if count == 10:
        break

Total reviews obtained: 200
Sample review (1): ‘The Attachment Diaries’ Review: Love, Sick
Sample review (2): Review: ‘What’s Love Got to Do With It?’ Probably a Lot
Sample review (3): ‘You Can Live Forever’ Review: Do You Love Me Now?
Sample review (4): ‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip
Sample review (5): ‘Other People’s Children’ Review: True Romance
Sample review (6): ‘One True Loves’ Review: A Romance Lost at Sea
Sample review (7): ‘The Lost Weekend: A Love Story’ Review: When John Lennon Strayed
Sample review (8): ‘A Thousand and One’ Review: A New York Love Story
Sample review (9): ‘Your Place or Mine’ Review: Try Neither
Sample review (10): ‘Love in the Time of Fentanyl’ Review: Heartbreak, Death and Hope in Vancouver


In [251]:
# Preview first 5 results in JSON format using json.dumps w/ argument indent=4 to format data
for i in range(5):
    print(f"\nJSON Sample {i} ({reviews_list[i]['headline']['main']})")
    print(json.dumps(reviews_list[i], indent=4))


JSON Sample 0 (‘The Attachment Diaries’ Review: Love, Sick)
{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
      

In [271]:
# Use json_normalize to convert all_results to a DataFrame
reviews_df = pd.json_normalize(reviews_list)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [272]:
# getting a list of columns
reviews_df.columns

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')

In [273]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
#   Title is between unicode characters \u2018 and \u2019. 
#   End string should include " Review" to avoid cutting title early

# str : str[start, stop]
# str.find(sub[, start[, end]] )  # sub: substring to be searched in the string.

reviews_df['title'] = reviews_df['headline.main'].apply(\
        lambda str: str[str.find("\u2018")+1 : str.find("\u2019 Review")])
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [274]:

# Fix the "keywords" column by converting cells from a list to a string:

#   Use `extract_keywords` function to convert the `"keywords"` column from 
#     a list of dictionaries to strings using the `apply()` method.

# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords


reviews_copy_df = reviews_df.copy()
reviews_copy_df['keywords'] = reviews_copy_df['keywords'].apply(extract_keywords)
reviews_copy_df.head()
        

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [275]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_edit_02_df['title'].to_list()
titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

In [277]:
print(f'total titles: {len(titles)}')
print(titles)

total titles: 200
['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan',

### Access The Movie Database API

In [278]:
# Prepare The Movie Database query
# https://developer.themoviedb.org/docs/search-and-query-for-details

url = "https://api.themoviedb.org/3/search/movie?query="
print(type(tmdb_api_key))
tmdb_key_string = "&api_key=" + tmdb_api_key


<class 'str'>


In [283]:
# SAMPLE QUERY, JSON dump for formatting investigation.

# Search:
#  issue a query to one of the movie, TV show or person search methods
# 'https://api.themoviedb.org/3/search/movie?query=Jack+Reacher&api_key=API_KEY
#   look at is the 'results' field. 
#   This is an array and will contain standard movie list objects.
query = 'The Attachment Diaries'
query_url = f"{url}{query}{tmdb_key_string}"
print(query_url)
sample_movie_request = requests.get(query_url).json() 
# Use json.dumps with argument indent=4 to format data
# print(json.dumps(sample_movie_request, indent=4))


https://api.themoviedb.org/3/search/movie?query=The Attachment Diaries&api_key=a78f3a912736e51a4277ddf596af9b2f
{
    "page": 1,
    "results": [
        {
            "adult": false,
            "backdrop_path": null,
            "genre_ids": [
                18,
                9648,
                53,
                27
            ],
            "id": 743040,
            "original_language": "es",
            "original_title": "El apego",
            "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
            "popularity": 4.816,
            "poster_path": "/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg",
            "release_date": "2021-10-07"

In [289]:
# SAMPLE get movie id from JSON object
sample_movie_id = sample_movie_request["results"][0]["id"]
print(movie_id)

# Make a request for a the full movie details using movie id
# url 'https://api.themoviedb.org/3/movie/343611?api_key=API_KEY'

# NEW QUERY URL !!
url_id = 'https://api.themoviedb.org/3/movie/'
id_query_url = f"{url_id}{sample_movie_id}?{tmdb_key_string}"
print(id_query_url)
# sample_movie_details_request = requests.get(id_query_url).json()
# print(json.dumps(sample_movie_details_request, indent=4))



743040
https://api.themoviedb.org/3/movie/743040?&api_key=a78f3a912736e51a4277ddf596af9b2f


In [290]:

# Create an empty list called tmdb_movies_list to store results from API requests. 
# This will contain a list of dictionaries.
tmdb_movies_list = []
genres = []
spoken_languages =[]
production_countries = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for t in titles:
    # Check if we need to sleep before making a request
    # Use time.sleep(1) when it reaches a multiple of 50 and display a msg
    if request_counter % 50 == 0:
        print(f'request counter: {request_counter}, commence sleep.')
        time.sleep(1)
    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie is not found.
    try:
        query_url = f"{url}{t}{tmdb_key_string}"
        movie_request = requests.get(query_url).json()  
    # Get movie id
        movie_id = sample_movie_request["results"][0]["id"]
        print(f'Found: {t}; id: {movie_id}')

    # Make a request for a the full movie details using movie id
    # Query For Details
    # url 'https://api.themoviedb.org/3/movie/343611?api_key=API_KEY'
    # NEW QUERY URL !!
        url_id = 'https://api.themoviedb.org/3/movie/'
        id_query_url = f"{url_id}{movie_id}?{tmdb_key_string}"
        print('https://api.themoviedb.org/3/movie/343611?api_key=API_KEY')
        print('my query: ', id_query_url)


        # id_query_url = f"{url}{movie_id}{tmdb_key_string}"
        # movie_details_request = requests.get(id_query_url).json()
        # print(json.dumps(movie_details_request, indent=4))

    except:
        print(f"Couldn't find response for Movie Title '{t}' in TMDB.")



# # If there are valid 'results':
# # # Extract the genre names from the results into a list called genres.
#         # print(len(movie_request['results']) > 0)
#         if len(movie_request['results']) > 0:
#             print(f"Found genre ids {movie_request['results']['genre_ids']} for {t}")
#             # genres = genres.append(movie_request['results']['genre_ids'])
        
#         else:
#             print(f"No movie Results {movie_request['results']} for {t}")
#             print(json.dumps(movie_request, indent=4))
        




Found: The Attachment Diaries; id: 743040
https://api.themoviedb.org/3/movie/343611?api_key=API_KEY
https://api.themoviedb.org/3/movie/743040?&api_key=a78f3a912736e51a4277ddf596af9b2f
Found: What’s Love Got to Do With It?’ Probably a Lo; id: 743040
https://api.themoviedb.org/3/movie/343611?api_key=API_KEY
https://api.themoviedb.org/3/movie/743040?&api_key=a78f3a912736e51a4277ddf596af9b2f
Found: You Can Live Forever; id: 743040
https://api.themoviedb.org/3/movie/343611?api_key=API_KEY
https://api.themoviedb.org/3/movie/743040?&api_key=a78f3a912736e51a4277ddf596af9b2f
Found: A Tourist’s Guide to Love; id: 743040
https://api.themoviedb.org/3/movie/343611?api_key=API_KEY
https://api.themoviedb.org/3/movie/743040?&api_key=a78f3a912736e51a4277ddf596af9b2f
Found: Other People’s Children; id: 743040
https://api.themoviedb.org/3/movie/343611?api_key=API_KEY
https://api.themoviedb.org/3/movie/743040?&api_key=a78f3a912736e51a4277ddf596af9b2f
Found: One True Loves; id: 743040
https://api.themovied

In [169]:
movies_request

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/wDwQXLDUuiEaaiuWIDBpbqnwYGX.jpg',
   'genre_ids': [18],
   'id': 492188,
   'original_language': 'en',
   'original_title': 'Marriage Story',
   'overview': 'A stage director and an actress struggle through a grueling, coast-to-coast divorce that pushes them to their personal extremes.',
   'popularity': 30.798,
   'poster_path': '/91osxUqsq1JC0zJzB1CpAavlWu7.jpg',
   'release_date': '2019-09-28',
   'title': 'Marriage Story',
   'video': False,
   'vote_average': 7.745,
   'vote_count': 6758},
  {'adult': False,
   'backdrop_path': None,
   'genre_ids': [10749],
   'id': 271794,
   'original_language': 'ko',
   'original_title': '결혼 이야기',
   'overview': "This film is a social comedy which depicts women's endeavor to get the right to live free from men's violence in a male dominated society. This Film begins with a newly married couple. They wed against the wishes of family and friends, who thought they weren't a good matc

In [ ]:



#  Extract the genre names from the results into a list called genres.
        genres[] = 

#  Extract the spoken_languages' English name from results to list spoken_languages.
        spoken_languages[]=

#  Extract the production_countries' name from results to list production_countries.
        production_countries = 

# Add the relevant data to a dictionary:
#  Create a dictionary with the following results: 
#      `title`, `original_title`, `budget`, `original_language`, 
#      `homepage`, `overview`, `popularity`, `runtime`, `revenue`, 
#      `release_date`, `vote_average`, `vote_count`, as well as the 
#      `genres`, `spoken_languages`, and `production_countries` lists you just created

        movie_dictionary = {   'title': , 
                                'original_title': , 
                                'budget': , 
                                'original_language': , 
                                'homepage': , 
                                'overview': , 
                                'popularity': , 
                                'runtime': , 
                                'revenue': , 
                                'release_date': , 
                                'vote_average': , 
                                'vote_count': , 
                                'genres': genres, 
                                'spoken_languages': spoken_languages , 
                                'production_countries': production_countries }

#  # append dictionary to `tmdb_movies_list`
        tmdb_movies_list.append()

#  Print out the name of the movie and a message to indicate that the title was found.
#  Print out the title that was found



      

In [145]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


# Preview first 5 results in JSON format using json.dumps w/ argument indent=4 to format data
for i in range(5):
    print(f"\nMovies JSON Sample {i} ({movies_request[i]['results']['id']})") #snippet
    print(json.dumps(movies_request[i], indent=4))


JSON Sample 0 (Review: ‘Kangaroo: A Love-Hate Story’ Exposes a Wildlife Massacre)
{
    "abstract": "The documentary looks at the mass killings of kangaroos for pet-food companies, leather processors and ranchers in Australia.",
    "web_url": "https://www.nytimes.com/2018/01/18/movies/kangaroo-a-love-hate-story-review.html",
    "snippet": "The documentary looks at the mass killings of kangaroos for pet-food companies, leather processors and ranchers in Australia.",
    "lead_paragraph": "Brutality overrides everything else in \u201cKangaroo: A Love-Hate Story\u201d \u2014 views of beautiful landscapes are hard to enjoy after seeing wild animals, including their young, maimed and slaughtered. But the filmmakers are determined to sound a wake-up siren, and they blast it here with extra strength.",
    "print_section": "C",
    "print_page": "9",
    "source": "The New York Times",
    "multimedia": [
        {
            "rank": 0,
            "subtype": "xlarge",
            "captio

In [ ]:
# Convert the results to a DataFrame


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,0,2021-10-07,4.000,2,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,0,2023-03-24,7.643,21,[Canada]
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy, Adventure]",en,[English],https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",25.369,96,0,2023-04-21,6.389,122,[United States of America]
3,Other People's Children,Les Enfants des autres,0,[Drama],fr,[French],https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",16.246,104,0,2022-09-21,6.900,137,[France]
4,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,37820,2023-04-07,6.681,36,"[Germany, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,0,2017-01-11,5.617,184,[United States of America]
151,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,0,2017-01-14,6.051,108,"[Belgium, France]"
152,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,0,2006-05-15,6.708,12,[United Kingdom]
153,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[English, Turkish]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",12.097,111,413844,2017-03-10,6.150,236,"[Turkey, United States of America]"


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[],,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,[Drama],en,[English],https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,In Search of Fellini,In Search of Fellini,0,"[Drama, Adventure]",en,[English],,"Lucy, a small-town girl from Ohio, discovers t...",4.103,93,...,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None
121,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
122,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
123,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [ ]:
# Export data to CSV without the index
